In [1]:
! pip install gensim

In [2]:
! pip install pyLDAvis

In [3]:
! pip install spacy

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import scipy
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.multivariate.pca import PCA
import numpy as np
import json
import glob
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

Slow version of gensim.models.doc2vec is being used
Slow version of Fasttext is being used


In [5]:
train_df = pd.read_csv('WikiLarge_Train.csv')
test_df = pd.read_csv('WikiLarge_Test.csv')
train_df#['original_text']

,original_text,label
0,There is manuscript evidence that Austen conti...,1
1,"In a remarkable comparative analysis , Mandaea...",1
2,"Before Persephone was released to Hermes , who...",1
3,Cogeneration plants are commonly found in dist...,1
4,"Geneva -LRB- , ; , ; , ; ; -RRB- is the second...",1
...,...,...
416763,A Duke Nukem 3D version has been sold for Xbox...,0
416764,"However , it is becoming replaced as a method ...",0
416765,There are hand gestures in both Hindu and Budd...,0
416766,"If it is necessary to use colors , try to choo...",0


In [6]:
# much of the following work will be heavily influenced by the following instructional video: https://youtu.be/UEn3xHNBXJU?list=LL

# Using LDA to train supervised learning model

In [7]:
# saved to csv
def lemmatize_sentence(sentences, parts_of_speech=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    sentences_out = []
    for sentence in tqdm(sentences):
        doc = nlp(sentence)
        new_sentence = []
        for token in doc:
            if token.pos_ in parts_of_speech:
                new_sentence.append(token.lemma_)
        final = " ".join(new_sentence)
        sentences_out.append(final)
    return(sentences_out)


lemmatized_sentences = lemmatize_sentence(train_df['original_text'])
print (lemmatized_sentences[0][0:90])

pd.DataFrame(lemmatized_sentences).to_csv('lemmatized_sentences.csv', index = False)

'def lemmatize_sentence(sentences, parts_of_speech=["NOUN", "ADJ", "VERB", "ADV"]):\n    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])\n    sentences_out = []\n    for sentence in tqdm(sentences):\n        doc = nlp(sentence)\n        new_sentence = []\n        for token in doc:\n            if token.pos_ in parts_of_speech:\n                new_sentence.append(token.lemma_)\n        final = " ".join(new_sentence)\n        sentences_out.append(final)\n    return(sentences_out)\n\n\nlemmatized_sentences = lemmatize_sentence(train_df[\'original_text\'])\nprint (lemmatized_sentences[0][0:90])\n\npd.DataFrame(lemmatized_sentences).to_csv(\'lemmatized_sentences.csv\', index = False)'

In [8]:
lemmatized_sentences = pd.read_csv('lemmatized_sentences.csv')
lemmatized_sentences

,0
0,manuscript evidence continue work piece as lat...
1,remarkable comparative analysis mandaean schol...
2,release send retrieve trick eat pomegranate se...
3,cogeneration plant commonly find district heat...
4,second most populous city zürich most populous...
...,...
416763,3d version sell live
416764,however become replace method execute criminal...
416765,hand gesture iconography
416766,necessary use color try choose color will conf...


In [9]:
lemmatized_sentences['0']

0         manuscript evidence continue work piece as lat...
1         remarkable comparative analysis mandaean schol...
2         release send retrieve trick eat pomegranate se...
3         cogeneration plant commonly find district heat...
4         second most populous city zürich most populous...
                                ...                        
416763                                 3d version sell live
416764    however become replace method execute criminal...
416765                             hand gesture iconography
416766    necessary use color try choose color will conf...
416767                                                  NaN
Name: 0, Length: 416768, dtype: object

In [10]:
nan_df = lemmatized_sentences[lemmatized_sentences.isna().any(axis=1)]
nan_df

,0
31,NaN
90,NaN
104,NaN
140,NaN
145,NaN
...,...
416690,NaN
416694,NaN
416730,NaN
416745,NaN


In [11]:
nan_df2 = train_df[train_df.isna().any(axis=1)]
nan_df2

,original_text,label


In [12]:
print(lemmatized_sentences['0'][0][0:90])

manuscript evidence continue work piece as late period niece nephew make further addition 


In [13]:
type(lemmatized_sentences)

pandas.core.frame.DataFrame

In [14]:
lemmatized_sentences['0'][40:50]

40    bear american professional wrestler actor well...
41                 situate northern border coastline km
42    shade red orange may sometimes appear replace ...
43    also know name take newspaper mean clarion tak...
44    guitar traditionally construct various wood st...
45    player take role engineer name battle polymorp...
46             rez bear also know mexican racing driver
47         order book bind book click order book button
48    conflict parent religion mean most baptise bir...
49                           ndash german mathematician
Name: 0, dtype: object

In [15]:
train_df['original_text'][30:35]

30    On July 29 , 2008 the Palace announced that Ki...
31                               F. W. D. Brie , 1906 .
32    Simon Boccanegra is an opera with a prologue a...
33    On June 6 , 2000 , ten days before the movie w...
34    The Bushehr Nuclear Power Plant -LRB- -RRB- is...
Name: original_text, dtype: object

In [16]:
lemmatized_sentences = lemmatized_sentences.fillna('nan')

In [17]:
def gen_words(sentences):
    final = []
    for sentence in tqdm(sentences['0']):
        new = gensim.utils.simple_preprocess(sentence, deacc=True)
        final.append(new)
    return (final)

clean_words = gen_words(lemmatized_sentences)

print (clean_words[0][0:20])

100%|██████████| 416768/416768 [00:08<00:00, 47271.73it/s]

['manuscript', 'evidence', 'continue', 'work', 'piece', 'as', 'late', 'period', 'niece', 'nephew', 'make', 'further', 'addition', 'as', 'late']


In [18]:
id2word = corpora.Dictionary(clean_words)

corpus = []
for sentence in tqdm(clean_words):
    new = id2word.doc2bow(sentence)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

100%|██████████| 416768/416768 [00:03<00:00, 118308.14it/s]

[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]
addition


In [19]:
# model saved
lda_model = gensim.models.ldamodel.LdaModel(corpus=tqdm(corpus),
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=40,
                                           update_every=1,
                                           chunksize=80,
                                           passes=8,
                                           alpha="auto")
                                           
lda_model.save("test_model.model")

'lda_model = gensim.models.ldamodel.LdaModel(corpus=tqdm(corpus),\n                                           id2word=id2word,\n                                           num_topics=20,\n                                           random_state=40,\n                                           update_every=1,\n                                           chunksize=80,\n                                           passes=8,\n                                           alpha="auto")\n                                           \nlda_model.save("test_model.model")'

In [20]:
saved_model = gensim.models.ldamodel.LdaModel.load("test_model.model")

In [21]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(saved_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
19     0.158146  0.150274       1        1  6.038866
15     0.151237 -0.382570       2        1  5.975503
3     -0.142601 -0.080259       3        1  5.863647
13    -0.083645  0.474917       4        1  5.807498
11     0.337261 -0.355894       5        1  5.694960
10     0.405806 -0.183368       6        1  5.478079
16     0.135771  0.454111       7        1  5.464071
4     -0.280999 -0.406879       8        1  5.179016
7     -0.465357  0.157534       9        1  5.149288
14     0.307618  0.347551      10        1  5.061494
12     0.434210  0.192198      11        1  4.951093
0     -0.059372  0.263811      12        1  4.950971
2     -0.354525 -0.156566      13        1  4.893123
8     -0.299726  0.381977      14        1  4.526108
18    -0.001323 -0.486032      15        1  4.444771
1     -0.289231  0.160079      16        1  4.390353
5     -0.112774 -0.320283      17        1  4.181380
9     -0.482125 -0.123240      18        1  4.044359
17     0.179678 -0.088160      19        1  4.026997
6      0.461952  0.000801      20        1  3.878423, topic_info=            Term          Freq         Total Category  logprob  loglift
279          nan  26152.000000  26152.000000  Default  30.0000  30.0000
350         bear  21098.000000  21098.000000  Default  29.0000  29.0000
371         also  22562.000000  22562.000000  Default  28.0000  28.0000
73         first  20316.000000  20316.000000  Default  27.0000  27.0000
231          use  21879.000000  21879.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
2266     cyclone   1106.972357   1108.086754  Topic20  -4.6459   3.2487
1132  individual   1075.422142   1076.536538  Topic20  -4.6748   3.2487
161        heart   1006.579891   1007.694288  Topic20  -4.7410   3.2486
392       battle    985.306344    986.420741  Topic20  -4.7623   3.2486
1567      method    979.977837    981.092233  Topic20  -4.7678   3.2486

[630 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1004     15  0.999198      able
837       9  0.999742     about
22       20  0.999635    accord
280      11  0.998898       act
1315      1  0.999128  activity
...     ...       ...       ...
125      11  0.999829     write
322      16  0.999504    writer
34        5  0.999883      year
370      18  0.998004    yellow
1867      6  0.999396     young

[600 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 16, 4, 14, 12, 11, 17, 5, 8, 15, 13, 1, 3, 9, 19, 2, 6, 10, 18, 7])

In [22]:
corpus[400001]

[(40, 1), (659, 1), (1669, 1), (2545, 1), (4508, 1)]

In [23]:
train_df['original_text'][400001]

"They decide to find Shade 's father , Cassiel , and Chinook volunteers to help ."

# Get bag of words for test df
Test DF code not used because the accuracy was too low with the train df train test split

In [24]:
test_sentence = ['This', 'is', 'a', 'test']
id2word.doc2bow(test_sentence)

[(632, 1), (2408, 1)]

In [25]:
# saved to csv
def lemmatize_sentence(sentences, parts_of_speech=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    sentences_out = []
    for sentence in tqdm(sentences):
        doc = nlp(sentence)
        new_sentence = []
        for token in doc:
            if token.pos_ in parts_of_speech:
                new_sentence.append(token.lemma_)
        final = " ".join(new_sentence)
        sentences_out.append(final)
    return(sentences_out)


lemmatized_sentences = lemmatize_sentence(test_df['original_text'])
print (lemmatized_sentences[0][0:90])

pd.DataFrame(lemmatized_sentences).to_csv('lemmatized_test_sentences.csv', index = False)

'def lemmatize_sentence(sentences, parts_of_speech=["NOUN", "ADJ", "VERB", "ADV"]):\n    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])\n    sentences_out = []\n    for sentence in tqdm(sentences):\n        doc = nlp(sentence)\n        new_sentence = []\n        for token in doc:\n            if token.pos_ in parts_of_speech:\n                new_sentence.append(token.lemma_)\n        final = " ".join(new_sentence)\n        sentences_out.append(final)\n    return(sentences_out)\n\n\nlemmatized_sentences = lemmatize_sentence(test_df[\'original_text\'])\nprint (lemmatized_sentences[0][0:90])\n\npd.DataFrame(lemmatized_sentences).to_csv(\'lemmatized_test_sentences.csv\', index = False)'

In [26]:
lemmatized_test_sentences = pd.read_csv('lemmatized_test_sentences.csv')
lemmatized_test_sentences = lemmatized_test_sentences.fillna('nan')

In [27]:
def gen_words(sentences):
    final = []
    for sentence in tqdm(sentences['0']):
        new = gensim.utils.simple_preprocess(sentence, deacc=True)
        final.append(new)
    return (final)

clean_words = gen_words(lemmatized_test_sentences)

print(clean_words[1][0:20])

100%|██████████| 119092/119092 [00:02<00:00, 51424.40it/s]

['nan']


In [28]:
for x in range(300):
    print(clean_words[x][0:20])

['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['feb']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['nan']
['direct', 'video', 'sequel', 'film']
['fact', 'excommunicate', 'leave', 'second', 'time', 'result', 'victory', 'mark', 'end']
['tv', 'series', 'base', 'famous', 'video', 'game', 'character']
['nan']
['shall', 'issue', 'probable', 'cause', 'support', 'affirmation', 'particularly', 'describe', 'place', 'search', 'person', 'thing', 'seize']
['represent', 'occasion', 'require', 'prompt', 'attendance', 'spot', 'keep', 'study', 'ignorance', 'part', 'concerted', 'national', 'rising', 'still', 'look', 'submissive', 'vassal']
['represent', 'occasion', 'require', 'prompt', 'attendance', 'spot', 'keep', 'study', 'ignorance', 'part', 'concerted', 'national', 'rising', 'still', 'look', 'submissive', 'vassal']
['home']
['life', 'alternativ

In [29]:
#don't use seperately for test set
'''id2word = corpora.Dictionary(clean_words)

corpus = []
for sentence in tqdm(clean_words):
    new = id2word.doc2bow(sentence)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print(word)'''

'id2word = corpora.Dictionary(clean_words)\n\ncorpus = []\nfor sentence in tqdm(clean_words):\n    new = id2word.doc2bow(sentence)\n    corpus.append(new)\n\nprint (corpus[0][0:20])\n\nword = id2word[[0][:1][0]]\nprint(word)'

In [30]:
clean_words[206]

['song',
 'perform',
 'american',
 'rapper',
 'hip',
 'hop',
 'artist',
 'feature',
 'vocal',
 'soul',
 'singer']

In [31]:
doc_bow_test = id2word.doc2bow(clean_words[30])
doc_lda_test = saved_model[doc_bow_test]
doc_lda_test[0][1]

0.04797438

In [32]:
for x in range(10):
    print(saved_model.get_document_topics(corpus, per_word_topics=True)[x][0])

[(0, 0.033260193), (1, 0.02897588), (2, 0.03274646), (3, 0.062044665), (4, 0.03576819), (5, 0.027169282), (6, 0.0477914), (7, 0.27583024), (8, 0.031017572), (9, 0.026658097), (10, 0.03653344), (11, 0.03762208), (12, 0.03327965), (13, 0.061430603), (14, 0.033766504), (15, 0.039203547), (16, 0.0372513), (17, 0.027752254), (18, 0.02939794), (19, 0.06250068)]
[(0, 0.063517116), (1, 0.033361092), (2, 0.03770231), (3, 0.04621118), (4, 0.04118135), (5, 0.05650441), (6, 0.029800825), (7, 0.040117778), (8, 0.035711773), (9, 0.030692535), (10, 0.06728574), (11, 0.043315805), (12, 0.038316194), (13, 0.045504186), (14, 0.038876727), (15, 0.045136612), (16, 0.19422887), (17, 0.03195228), (18, 0.033847027), (19, 0.046736203)]
[(0, 0.034657605), (1, 0.07469106), (2, 0.057201073), (3, 0.041730534), (4, 0.037267808), (5, 0.028388958), (6, 0.16614671), (7, 0.05931876), (8, 0.03235824), (9, 0.027871639), (10, 0.038031105), (11, 0.108059354), (12, 0.034673672), (13, 0.041108556), (14, 0.036662206), (15, 0

# Text vector for train data

In [33]:
# saved to csv
topic_count = 20

for y in range(topic_count):
    train_df['sentence_vectors_'+str(y)] = .00001

for x in tqdm(range(len(train_df))):
    for y in range(topic_count):
        train_df['sentence_vectors_'+str(y)][x] = saved_model.get_document_topics(corpus)[x][y][1]
        
train_df.to_csv('train_df_vectors.csv', index = False)

"topic_count = 20\n\nfor y in range(topic_count):\n    train_df['sentence_vectors_'+str(y)] = .00001\n\nfor x in tqdm(range(len(train_df))):\n    for y in range(topic_count):\n        train_df['sentence_vectors_'+str(y)][x] = saved_model.get_document_topics(corpus)[x][y][1]\n        \ntrain_df.to_csv('train_df_vectors.csv', index = False)"

In [34]:
train_df

,original_text,label
0,There is manuscript evidence that Austen conti...,1
1,"In a remarkable comparative analysis , Mandaea...",1
2,"Before Persephone was released to Hermes , who...",1
3,Cogeneration plants are commonly found in dist...,1
4,"Geneva -LRB- , ; , ; , ; ; -RRB- is the second...",1
...,...,...
416763,A Duke Nukem 3D version has been sold for Xbox...,0
416764,"However , it is becoming replaced as a method ...",0
416765,There are hand gestures in both Hindu and Budd...,0
416766,"If it is necessary to use colors , try to choo...",0


In [35]:
train_df_vectors = pd.read_csv('train_df_vectors.csv')

train_df_vectors

,original_text,label,sentence_vectors_0,sentence_vectors_1,sentence_vectors_2,sentence_vectors_3,sentence_vectors_4,sentence_vectors_5,sentence_vectors_6,sentence_vectors_7,...,sentence_vectors_10,sentence_vectors_11,sentence_vectors_12,sentence_vectors_13,sentence_vectors_14,sentence_vectors_15,sentence_vectors_16,sentence_vectors_17,sentence_vectors_18,sentence_vectors_19
0,There is manuscript evidence that Austen conti...,1,0.033260,0.028976,0.032746,0.062045,0.035768,0.027169,0.047791,0.275830,...,0.036533,0.037622,0.033280,0.061431,0.033767,0.039204,0.037251,0.027752,0.029398,0.062501
1,"In a remarkable comparative analysis , Mandaea...",1,0.063517,0.033361,0.037702,0.046211,0.041181,0.056504,0.029801,0.040118,...,0.067286,0.043316,0.038316,0.045504,0.038877,0.045137,0.194229,0.031952,0.033847,0.046736
2,"Before Persephone was released to Hermes , who...",1,0.034658,0.074690,0.057201,0.041731,0.037268,0.028389,0.166147,0.059319,...,0.038031,0.108059,0.034674,0.041109,0.036664,0.040790,0.038791,0.029030,0.030990,0.042231
3,Cogeneration plants are commonly found in dist...,1,0.027814,0.044129,0.027832,0.033362,0.030259,0.025108,0.043578,0.029503,...,0.049997,0.031730,0.027817,0.069963,0.065734,0.311604,0.031428,0.023435,0.024740,0.053420
4,"Geneva -LRB- , ; , ; , ; ; -RRB- is the second...",1,0.038199,0.033355,0.215813,0.045976,0.041075,0.031306,0.029820,0.040028,...,0.041910,0.043170,0.038216,0.095851,0.038786,0.044946,0.042749,0.032000,0.033853,0.046533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416763,A Duke Nukem 3D version has been sold for Xbox...,0,0.074844,0.039310,0.044426,0.054452,0.048525,0.036859,0.035115,0.047272,...,0.049563,0.051040,0.045149,0.083340,0.045810,0.053186,0.050537,0.037650,0.039883,0.084792
416764,"However , it is becoming replaced as a method ...",0,0.039285,0.034224,0.038678,0.047407,0.068123,0.032091,0.056448,0.041156,...,0.043151,0.044437,0.039308,0.046682,0.039883,0.098057,0.095751,0.032779,0.060599,0.047946
416765,There are hand gestures in both Hindu and Budd...,0,0.046229,0.040336,0.045630,0.055712,0.049806,0.037828,0.035976,0.081741,...,0.050763,0.052356,0.046239,0.054903,0.046964,0.087202,0.051841,0.038714,0.040959,0.056473
416766,"If it is necessary to use colors , try to choo...",0,0.028838,0.044118,0.066382,0.110780,0.031012,0.042552,0.022442,0.030211,...,0.031676,0.089604,0.028855,0.053263,0.067266,0.033991,0.051293,0.024062,0.025489,0.035195


In [36]:
#saved_model.get_document_topics(corpus, per_word_topics=False)[0]#[0][0][1]


In [37]:
#saved_model.get_document_topics(corpus, per_word_topics=True)[0]#[0][0][1]

In [38]:
#doc_bow_test = id2word.doc2bow(clean_words[50])
#doc_lda_test = saved_model[doc_bow_test]
#doc_lda_test[0][1][1]

# Text vector for test df
Test DF code not used because the accuracy was too low with the train df train test split

In [39]:
# saved to csv
topic_count = 20

for y in range(topic_count):
    test_df['sentence_vectors_'+str(y)] = .01

for x in tqdm(range(len(test_df))):
    doc_bow_test = id2word.doc2bow(clean_words[x])
    doc_lda_test = saved_model[doc_bow_test]        
    for y in range(topic_count):
        test_df['sentence_vectors_'+str(y)][x] = doc_lda_test[y][1]
        
test_df.to_csv('test_df_vectors.csv', index = False)

"topic_count = 20\n\nfor y in range(topic_count):\n    test_df['sentence_vectors_'+str(y)] = .01\n\nfor x in tqdm(range(len(test_df))):\n    doc_bow_test = id2word.doc2bow(clean_words[x])\n    doc_lda_test = saved_model[doc_bow_test]        \n    for y in range(topic_count):\n        test_df['sentence_vectors_'+str(y)][x] = doc_lda_test[y][1]\n        \ntest_df.to_csv('test_df_vectors.csv', index = False)"

In [40]:
test_df_vectors = pd.read_csv('test_df_vectors.csv')

test_df_vectors

,id,original_text,label,sentence_vectors_0,sentence_vectors_1,sentence_vectors_2,sentence_vectors_3,sentence_vectors_4,sentence_vectors_5,sentence_vectors_6,...,sentence_vectors_10,sentence_vectors_11,sentence_vectors_12,sentence_vectors_13,sentence_vectors_14,sentence_vectors_15,sentence_vectors_16,sentence_vectors_17,sentence_vectors_18,sentence_vectors_19
0,0,-2011,NaN,0.047974,0.041795,0.047233,0.057893,0.083192,0.039189,0.037334,...,0.052696,0.054266,0.048002,0.057008,0.048705,0.056547,0.053731,0.040030,0.042403,0.058551
1,1,-2011,NaN,0.047974,0.041795,0.047233,0.057893,0.083192,0.039189,0.037334,...,0.052696,0.054266,0.048002,0.057008,0.048705,0.056547,0.053731,0.040030,0.042403,0.058551
2,2,-2000,NaN,0.047974,0.041795,0.047233,0.057893,0.083192,0.039189,0.037334,...,0.052696,0.054266,0.048002,0.057008,0.048705,0.056547,0.053731,0.040030,0.042403,0.058551
3,3,-1997,NaN,0.047974,0.041795,0.047233,0.057893,0.083192,0.039189,0.037334,...,0.052696,0.054266,0.048002,0.057008,0.048705,0.056547,0.053731,0.040030,0.042403,0.058551
4,4,1.636,NaN,0.047974,0.041795,0.047233,0.057893,0.083192,0.039189,0.037334,...,0.052696,0.054266,0.048002,0.057008,0.048705,0.056547,0.053731,0.040030,0.042403,0.058551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119087,119087,#NAME?,NaN,0.049540,0.043158,0.048775,0.059782,0.053275,0.040468,0.038553,...,0.054415,0.056037,0.049569,0.058868,0.050294,0.058392,0.055484,0.041336,0.043787,0.060462
119088,119088,#NAME?,NaN,0.049540,0.043158,0.048775,0.059782,0.053275,0.040468,0.038553,...,0.054415,0.056037,0.049569,0.058868,0.050294,0.058392,0.055484,0.041336,0.043787,0.060462
119089,119089,#NAME?,NaN,0.049540,0.043158,0.048775,0.059782,0.053275,0.040468,0.038553,...,0.054415,0.056037,0.049569,0.058868,0.050294,0.058392,0.055484,0.041336,0.043787,0.060462
119090,119090,#NAME?,NaN,0.049540,0.043158,0.048775,0.059782,0.053275,0.040468,0.038553,...,0.054415,0.056037,0.049569,0.058868,0.050294,0.058392,0.055484,0.041336,0.043787,0.060462


# Logistic Regression

In [41]:

X = train_df_vectors[['sentence_vectors_0',
                    'sentence_vectors_1',
                    'sentence_vectors_2',
                    'sentence_vectors_3',
                    'sentence_vectors_4',
                    'sentence_vectors_5',
                    'sentence_vectors_6',
                    'sentence_vectors_7',
                    'sentence_vectors_8',
                    'sentence_vectors_9',
                    'sentence_vectors_10',
                    'sentence_vectors_11',
                    'sentence_vectors_12',
                    'sentence_vectors_13',
                    'sentence_vectors_14',
                    'sentence_vectors_15',
                    'sentence_vectors_16',
                    'sentence_vectors_17',
                    'sentence_vectors_18',
                    'sentence_vectors_19',]]
y = train_df_vectors['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
X_train

,sentence_vectors_0,sentence_vectors_1,sentence_vectors_2,sentence_vectors_3,sentence_vectors_4,sentence_vectors_5,sentence_vectors_6,sentence_vectors_7,sentence_vectors_8,sentence_vectors_9,sentence_vectors_10,sentence_vectors_11,sentence_vectors_12,sentence_vectors_13,sentence_vectors_14,sentence_vectors_15,sentence_vectors_16,sentence_vectors_17,sentence_vectors_18,sentence_vectors_19
51137,0.046505,0.040514,0.045787,0.086752,0.050012,0.037988,0.036191,0.048720,0.043369,0.037274,0.051082,0.052604,0.046532,0.055261,0.047213,0.054815,0.082717,0.038804,0.041105,0.056757
142066,0.047974,0.041795,0.047233,0.057893,0.051592,0.039189,0.037334,0.050259,0.044740,0.038452,0.052696,0.054266,0.048002,0.057008,0.048705,0.056547,0.085331,0.040030,0.042403,0.058551
139053,0.064693,0.033877,0.038901,0.046643,0.042505,0.031792,0.031338,0.041263,0.037806,0.031239,0.042513,0.045501,0.038861,0.045976,0.068458,0.077812,0.086263,0.032798,0.060707,0.101056
167558,0.038294,0.033361,0.037702,0.046211,0.041181,0.056504,0.055024,0.065341,0.035712,0.030693,0.092509,0.043316,0.038316,0.045504,0.064100,0.095583,0.068112,0.031952,0.033847,0.046736
14327,0.052829,0.027717,0.052398,0.038247,0.055206,0.110828,0.024775,0.033278,0.071887,0.025534,0.055880,0.056928,0.031774,0.058670,0.095468,0.037383,0.056581,0.047786,0.028128,0.038704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67165,0.078868,0.029575,0.033394,0.040801,0.058908,0.118326,0.049080,0.035504,0.031635,0.027248,0.037183,0.083189,0.033900,0.062596,0.034404,0.062296,0.037925,0.073665,0.030014,0.041489
345309,0.064942,0.033929,0.038271,0.046600,0.041708,0.031868,0.030351,0.066600,0.036314,0.058316,0.043404,0.043810,0.038805,0.045924,0.039391,0.071361,0.043500,0.032589,0.061042,0.131274
122276,0.034689,0.053423,0.057265,0.064691,0.037302,0.028419,0.050276,0.036349,0.078586,0.051126,0.107026,0.039207,0.034705,0.064085,0.035222,0.040824,0.061815,0.029046,0.030733,0.065212
368860,0.113964,0.033361,0.037702,0.046211,0.041181,0.031281,0.029801,0.040118,0.035712,0.030693,0.042062,0.093762,0.063540,0.045504,0.038877,0.045137,0.042889,0.031952,0.109517,0.046736


In [42]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score


clf = LogisticRegression(solver='lbfgs', multi_class='auto')
#clf.fit(X_train[0:10000], y_train[0:10000])
clf.fit(X_train, y_train)

dummy_clf_uniform = DummyClassifier(strategy="uniform")
dummy_clf_uniform.fit(X_train, y_train)

dummy_clf_frequent = DummyClassifier(strategy="most_frequent")
dummy_clf_frequent.fit(X_train, y_train)

lr_tiny_preds = clf.predict(X_test)
rand_preds = dummy_clf_uniform.predict(X_test)
mf_preds = dummy_clf_frequent.predict(X_test)

lr_f1 = f1_score(y_test, lr_tiny_preds, average='macro')
rand_f1 = f1_score(y_test, rand_preds, average='macro')
mf_f1 = f1_score(y_test, mf_preds, average='macro')

print(lr_f1)
print(rand_f1)
print(mf_f1)

0.5372328647491273
0.4987050540192271
0.33246200763691347


In [43]:
import tensorflow as tf
from tensorflow import keras

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(20, activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Dense(10)
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping( monitor='val_loss', patience=30 )
lr_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=.5, patience=10, verbose=1 )

model.fit(X_train, y_train, epochs=5)

model.evaluate(X_test,  y_test, verbose=2)

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

9768/9768 [==============================] - 7s 673us/step - loss: 0.6994 - accuracy: 0.5365
Epoch 2/5
9768/9768 [==============================] - 6s 664us/step - loss: 0.6785 - accuracy: 0.5691
Epoch 3/5
9768/9768 [==============================] - 7s 686us/step - loss: 0.6737 - accuracy: 0.5801
Epoch 4/5
9768/9768 [==============================] - 7s 673us/step - loss: 0.6705 - accuracy: 0.5857
Epoch 5/5
9768/9768 [==============================] - 6s 648us/step - loss: 0.6687 - accuracy: 

[0.6679732203483582, 0.5934236645698547]